In [ ]:
import os    

from datetime import datetime

import sys
sys.path.append("..")

from algorithms.offline.any_percent_bc import get_actor_from_checkpoint as get_actor_from_checkpoint_bc, eval_actor as eval_actor_bc
get_actor_from_checkpoint = {
    "BC": get_actor_from_checkpoint_bc,
}
eval_actor = {
    "BC": eval_actor_bc,
}

from algorithms.utils.wrapper_gym import get_env

In [ ]:
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["MUJOCO_GL"] = "egl"

In [ ]:
MODEL = "BC"
PATH_MODEL = "../checkpoints/BC/BC-Go1JoystickFlatTerrain-72b53db8"
ENV = "Go1JoystickFlatTerrain"
N_EPISODES = 5
SEED = 1
RENDER = True
DEVICE = "cuda"

In [ ]:
render_trajectory = []

def render_callback(_, state):
    render_trajectory.append(state)

env_wrapped = get_env(ENV, "cuda", render_callback)

state_dim = env_wrapped.observation_space.shape[0] 
action_dim = env_wrapped.action_space.shape[0]
max_action = 1.0

In [ ]:
actor = get_actor_from_checkpoint[MODEL](checkpoint_path=PATH_MODEL, state_dim=state_dim, action_dim=action_dim, max_action=max_action)
episode_rewards = eval_actor[MODEL](actor=actor, env=env_wrapped, device=DEVICE, n_episodes=N_EPISODES, seed=SEED, render=RENDER)

print("Reward: ", episode_rewards.mean(), episode_rewards.std())

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
env_wrapped.save_video(render_trajectory, save_path=f"{PATH_MODEL}/{ENV}-{timestamp}.mp4")